Library Importing

In [1]:
import numpy as np
import scipy
import cv2
from seaborn import clustermap
import sklearn
import librosa
import glob
import os
import random
import json
import deepdish as dd
import matplotlib.pyplot as plt
%matplotlib inline

Compute downsampled Laplacian from Work and Performance name.
From http://librosa.github.io/librosa_gallery/auto_examples/plot_segmentation.html

In [ ]:
def compute_resampled_laplacian(W, P):
    
    #compose the file path from W, P
    crema_path = "./da-tacos_benchmark_subset_crema/" + W + "_crema/" + P + "_crema.h5"
    mfcc_path = "./da-tacos_benchmark_subset_mfcc/" + W + "_mfcc/" + P + "_mfcc.h5"
    
    #load h5 file
    crema = np.transpose(dd.io.load(crema_path)["crema"])
    mfcc = dd.io.load(mfcc_path)["mfcc_htk"]
    
    print(crema.shape)
    print(mfcc.shape)
    
    #------------------------------------------------------------------------------# 
    
    #Normally, when we had audio files, we would compute beat event locations to sync with cqt and mfccs
    #We don't have any beat data available for datacos (and can't compute them because we don't have the audio time series)
    
        #Beat tracking
        #tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
    
        #Beat-synchronize crema
        #Csync = librosa.util.sync(crema, beats, aggregate=np.median)
        
        #Beat-synchronize mfcc
        #Msync = librosa.util.sync(mfcc, beats)
        
        
    #Do I do some other form of dimensionality reduction (e.g. with 2D interpolation) and proceed as normal?
    #Do I not do any dimensionality reduction at this stage, and only care about downsampling the Laplacian at the end?
    #If the second question is true, I am confused as to why the shapes of crema and mfccs are always slightly different
    #e.g. crema is (12, 25784) while mfcc is (13, 25742) -of course I'm reffering to the second number
    #and what do I do to be able to make the later calculations (like for 'mu') that require same size properly 
        
    #------------------------------------------------------------------------------#    
    
    #Short-term history embedding
    Cstack = librosa.feature.stack_memory(crema, 4)

    #Building weighted recurrence matrix
    R = librosa.segment.recurrence_matrix(Cstack, width=3, mode='affinity', sym=True)
    
    #Enchancing diagonals with median filter
    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    
    #Multiangle path enchancement
    Rf = librosa.segment.path_enhance(Rf, 15)
    
    #Building sequence matrix using mfcc-similarity
    path_distance = np.sum(np.diff(mfcc, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance/sigma)
    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)
    
    #Compute balanced combination
    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)
    print(path_sim.shape)
    print(R_path.shape)
    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)
    A = mu * Rf + (1 - mu) * R_path
    
    #Plotting
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 3, 1)
    librosa.display.specshow(Rf, cmap='inferno_r', y_axis='time', y_coords=beat_times)
    plt.title('Recurrence similarity')
    plt.subplot(1, 3, 2)
    librosa.display.specshow(R_path, cmap='inferno_r')
    plt.title('Path similarity')
    plt.subplot(1, 3, 3)
    librosa.display.specshow(A, cmap='inferno_r')
    plt.title('Combined graph')
    plt.tight_layout()
    
compute_resampled_laplacian("W_18", "P_83070")

(12, 25784)
(13, 25742)


Compute Shape DNA from Laplacian.

In [ ]:
def compute_eigenvalues(L):
    

Compute set of multiple spectral clustering components of Laplacian.

In [ ]:
def compute_clustering_set(L):
    

Create dictionary with Shape DNAs and sets of multiple spectral clustering components of Laplacian from works with at least one performance.

In [ ]:
with open('./da-tacos_metadata/da-tacos_benchmark_subset_metadata.json') as f:
    benchmark_metadata = json.load(f)

dict = {}

for W in benchmark_metadata.keys():
    if len(benchmark_metadata[W].keys()) > 1:
        W_dict = {}
        for P in benchmark_metadata[W].keys():
            #Computations
            L_rs = compute_resampled_laplacian(W, P)
            shapeDNA = compute_eigenvalues(L_rs)
            L_set = compute_clustering_set(L_rs)
            
            #W Dictionary entry for P
            W_dict[P] = [shapeDNA, L_set]
        dict[W] = W_dict
        